	ber-language

	<program>		::= DISAS <var-decls> <statements> .leo

	<var-decls>		::= <var-decl> |
					<var-decl> <var-decls>

	<var-decl>		::= <type> REGISTER ;

				
	<type> 			::= long | int | hex | bin | ;

	<statements>		::= <open_statement> |
					<open_statement> <statements>

	<statement>		::= <atrib> |
					<if-else>

	<atrib>			::= REGISTER = <expression> ;

	<expression>		::= REGISTER |
					number |
					addw <expression>, <expression> |
					subw <expression>, <expression> |
					mul <expression>, <expression> |
					div <expression>, <expression> |
					( <expression> )





	<open_statement>	::= <atrib> |
				se-o-ber ( <expression> <comp> <expression> ){ <open_statement> } |
				se-o-ber ( <expression> <comp> <expression> ){ <closed_statement> } senao-o-ber { <open_statement> }



	<closed_statement>	::= <atrib> |
				se-o-ber ( <expression> <comp> <expression> ){ <closed_statement> } senao-o-ber { <closed_statement> }



					== |  != |   <   |   >  |     <=    |    >=
	<comp>			::= leo | ber | lucca | enzo | lucca-leo | enzo-leo





In [1]:
!pip install rply --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('BINUMBER', r'0b[01]+')
lg.add('HEXNUMBER', r'0x[0-9A-Fa-f]+')
lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')
lg.add('OPEN_BRACKET', r'\{')
lg.add('CLOSE_BRACKET', r'\}')

lg.add('INT', r'int')
lg.add('LONG', r'long')
lg.add('BIN', r'bin')
lg.add('HEX', r'hex')
lg.add('IF', r'se-o-ber')
lg.add('ELSE', r'senao-o-ber')
lg.add('WHILE', r'enquanto-o-ber')

lg.add('START', 'disas')
lg.add('END', '\.leo')
lg.add('COMP','leo')
lg.add('COMP','ber')
lg.add('COMP','enzo-leo')
lg.add('COMP','enzo')
lg.add('COMP','lucca-leo')
lg.add('COMP','lucca')
lg.add('REGISTER', r'[a-zA-Z][a-zA-Z0-9]*')

lg.add('EQUALS', r'=')
lg.add('SEMICOL', r';')

lg.ignore('\s+')

lexer = lg.build()

In [3]:
program = """
disas
    bin REGISTER1;

    REGISTER1 = 0b101010;
.leo
"""



a = lexer.lex(program)

for i in a:
    print(i)

Token('START', 'disas')
Token('BIN', 'bin')
Token('REGISTER', 'REGISTER1')
Token('SEMICOL', ';')
Token('REGISTER', 'REGISTER1')
Token('EQUALS', '=')
Token('BINUMBER', '0b101010')
Token('SEMICOL', ';')
Token('END', '.leo')


In [4]:
#ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITORS

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp):
        self.id = id
        self.tp = tp


    def accept(self, visitor):
        visitor.visit_vardecl(self)

class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)


class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_while(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass


class Sub(BinaryOp):
  pass


class Mul(BinaryOp):
  pass


class Div(BinaryOp):
  pass
